In [13]:
import pandas as pd
import psycopg2
import os

folder_path = 'C:/Users/moham/Desktop/Data Analysis/2025 Data Analysis/SQL Zero - Hero/Walmart'

csv_files = [
    ('Clean_Walmart_data_1.csv', 'walmart_data')
]

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='Walmart',
        port='5432'
    )
    cursor = conn.cursor()
    print("✅ Connected to the database.")

    for csv_file, table_name in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        print(f"📂 Trying to read file: {file_path}")

        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            continue

        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        df = df.where(pd.notnull(df), None)
        df.columns = [col.strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns});'
        cursor.execute(create_table_query)
        print(f"🛠️ Table `{table_name}` created or already exists.")

        # ✅ Fix: Use separate strings instead of nested f-strings
        column_names = ', '.join([f'"{col}"' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f'INSERT INTO "{table_name}" ({column_names}) VALUES ({placeholders});'

        for row in df.itertuples(index=False, name=None):
            cursor.execute(insert_query, row)

        conn.commit()
        print(f"✅ Data from `{csv_file}` inserted into `{table_name}`.")

except psycopg2.Error as err:
    print(f"❗Database error: {err}")
except Exception as e:
    print(f"❗General error: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
    print("🔒 Database connection closed.")




✅ Connected to the database.
📂 Trying to read file: C:/Users/moham/Desktop/Data Analysis/2025 Data Analysis/SQL Zero - Hero/Walmart\Clean_Walmart_data_1.csv
🛠️ Table `walmart_data` created or already exists.
✅ Data from `Clean_Walmart_data_1.csv` inserted into `walmart_data`.
🔒 Database connection closed.
